## THIS NOTEBOOK WILL NOT WORK IN GOOGLE COLAB!!! IT MUST BE RUN LOCALLY!!! You can download it as an .ipynb file and open that with Jupyter notebook/labs

In [122]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [123]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd

If you get an error saying something like "sqlalchemy not installed", you should add a code cell and write `pip install sqlalchemy`. Then, rerun the above cell (you might also need to install pymysql the same way)

In [124]:
password = 'Warning//2000'
db_name = 'music'

In [125]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password=password, database=db_name)

In [126]:
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

In [127]:
songs = pd.read_csv('songs.csv')
albums = pd.read_csv('albums.csv')
artists = pd.read_csv('artists.csv')
playlists = pd.read_csv('playlists.csv')
users = pd.read_csv('users.csv')

In [128]:
table_name = 'songs'
table_name1 = 'albums'
table_name2 = 'artists'
table_name3 = 'playlists'
table_name4 = 'users'

In [129]:
songs.to_sql(name=table_name, con=connection, if_exists='replace', index=False)
albums.to_sql(name=table_name1, con=connection, if_exists='replace', index=False)
artists.to_sql(name=table_name2, con=connection, if_exists='replace', index=False)
playlists.to_sql(name=table_name3, con=connection, if_exists='replace', index=False)
users.to_sql(name=table_name4, con=connection, if_exists='replace', index=False)

10

We use SQL syntax for querying. The triple quotation mark syntax allows us to write commands simiar to how we did in SQL

In [130]:
query1 = "SELECT * FROM users"

In [131]:
query2 = """

SELECT * FROM songs
WHERE Artist_ID = 8

"""

In [132]:
result_df = pd.read_sql(query1, con=connection)

In [133]:
result_df

,Creator_ID,Username,Country,Top_Song_ID,Top_Album_ID,Top_Artist_ID,Best_friend_ID
0,1,Blink21,USA,9.0,2.0,8.0,3.0
1,2,JaxWave,USA,NaN,NaN,NaN,4.0
2,3,LunaPix,Canada,16.0,9.0,2.0,6.0
3,4,ZorbX,USA,NaN,NaN,NaN,7.0
4,5,FizzDart,Mexico,34.0,2.0,6.0,NaN
5,6,QuikFox,Canada,9.0,2.0,8.0,3.0
6,7,HexRyn,Sweden,19.0,NaN,1.0,8.0
7,8,VimJet,Ireland,1.0,1.0,1.0,2.0
8,9,KyroM,USA,36.0,2.0,8.0,8.0
9,10,WispFiz,USA,54.0,11.0,15.0,6.0


In [134]:
result_df_2 = pd.read_sql(query2, con=connection)

In [135]:
result_df_2

,Song_ID,Title,Artist_ID,Album_ID,Playlist_ID,Genre,Rating,Duration
0,9,Golden,8.0,2.0,1.0,Pop,7,3:50
1,10,Adore You,8.0,2.0,5.0,Pop,7,4:25
2,36,Watermelon Sugar,8.0,2.0,2.0,Pop,9,2:54
3,37,Sign of the Times,8.0,2.0,4.0,Rock,9,5:40
4,38,Adore You (Live),8.0,2.0,3.0,Pop,8,3:27
5,39,Falling,8.0,2.0,7.0,Pop,8,4:00
6,40,Golden (Live),8.0,2.0,1.0,Pop,8,3:28


We can merge in PyMySQL just like in regular SQL

In [136]:
query3 = """
  SELECT users.Username, users.Country, songs.Title AS Fav_Song, songs.Genre
  FROM users
  LEFT JOIN songs ON users.Top_Song_ID = songs.Song_ID
"""

In [137]:
result_df_3 = pd.read_sql(query3, con=connection)

In [138]:
result_df_3

,Username,Country,Fav_Song,Genre
0,Blink21,USA,Golden,Pop
1,JaxWave,USA,None,None
2,LunaPix,Canada,Two-Headed Boy,Indie
3,ZorbX,USA,None,None
4,FizzDart,Mexico,My Worst Enemy,Pop
5,QuikFox,Canada,Golden,Pop
6,HexRyn,Sweden,Vampire,Pop
7,VimJet,Ireland,Happier,Pop
8,KyroM,USA,Watermelon Sugar,Pop
9,WispFiz,USA,Stuck In The Middle,Pop


In [139]:
###MONGO

In [140]:
import pymongo
from pymongo import MongoClient
import pandas as pd

In [141]:
client = MongoClient('localhost', 27017)
client
db = client.example
#collection = db.music

In [142]:
#users = pd.read_csv('users.csv')
users

,Creator_ID,Username,Country,Top_Song_ID,Top_Album_ID,Top_Artist_ID,Best_friend_ID
0,1,Blink21,USA,9.0,2.0,8.0,3.0
1,2,JaxWave,USA,NaN,NaN,NaN,4.0
2,3,LunaPix,Canada,16.0,9.0,2.0,6.0
3,4,ZorbX,USA,NaN,NaN,NaN,7.0
4,5,FizzDart,Mexico,34.0,2.0,6.0,NaN
5,6,QuikFox,Canada,9.0,2.0,8.0,3.0
6,7,HexRyn,Sweden,19.0,NaN,1.0,8.0
7,8,VimJet,Ireland,1.0,1.0,1.0,2.0
8,9,KyroM,USA,36.0,2.0,8.0,8.0
9,10,WispFiz,USA,54.0,11.0,15.0,6.0


In [143]:
merged_df = pd.merge(playlists, songs, on='Playlist_ID', how='left')
playlist_songs = merged_df.groupby('Playlist_ID')['Title'].apply(list).reset_index()
playlists_df = pd.merge(playlists, playlist_songs, on='Playlist_ID', how='left')
playlists_df['Title'] = playlists_df['Title'].apply(lambda x: x if isinstance(x, list) else [])
playlists_df = playlists_df.rename(columns={'Title': 'Songs_in_playlist'})
print(playlists_df.head(5))

   Playlist_ID  Playlist_name  Creator_ID                    Description  \
0            1   Driving Time           1             Songs to drive to.   
1            2            Gym           2             Songs for the gym.   
2            3   Cooking Time           4        Songs for cooking time.   
3            4          Happy           3  When you have had a GREAT day   
4            5  cleaning time           6                    bops fr fr    

   Popularity                                  Songs_in_playlist  
0        1000  [Happier, Golden , Love to Dream, Yellow, Gold...  
1        2500  [White Horse, High Hopes, Watermelon Sugar, Sh...  
2        1200  [Two-Headed Boy, Vampire, Love Like This, All ...  
3           0  [Enough for You, When We Were Young , Baby for...  
4           1  [Kiss Me More, Adore You, All Too Well, Love S...  


In [144]:
users_dict = users.to_dict('records')
users_dict
songs_dict = songs.to_dict('records')
albums_dict = albums.to_dict('records')
playlists_dict = playlists_df.to_dict('records')
artists_dict = artists.to_dict('records')

In [145]:
users_collection = db.users
albums_collection = db.albums
artists_collection = db.artists
playlists_collection = db.playlists
songs_collection = db.songs

In [146]:
users_collection.insert_many(users_dict)
albums_collection.insert_many(albums_dict)
songs_collection.insert_many(songs_dict)
artists_collection.insert_many(artists_dict)
playlists_collection.insert_many(playlists_dict)

InsertManyResult([ObjectId('665636bc5bcb0c8eb3c0e45f'), ObjectId('665636bc5bcb0c8eb3c0e460'), ObjectId('665636bc5bcb0c8eb3c0e461'), ObjectId('665636bc5bcb0c8eb3c0e462'), ObjectId('665636bc5bcb0c8eb3c0e463'), ObjectId('665636bc5bcb0c8eb3c0e464'), ObjectId('665636bc5bcb0c8eb3c0e465'), ObjectId('665636bc5bcb0c8eb3c0e466'), ObjectId('665636bc5bcb0c8eb3c0e467'), ObjectId('665636bc5bcb0c8eb3c0e468')], acknowledged=True)

In [121]:
### dont run until the end

users_collection.delete_many({})
songs_collection.delete_many({})
albums_collection.delete_many({})
artists_collection.delete_many({})
playlists_collection.delete_many({})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)